# __LOG6302A TP6__

## Détection de clones et similarités

-Manu Serpette : 2229693 <br>  
-Kaoukeb Ryan : 2228319

In [21]:
import os
from code_analysis import ASTReader
import numpy as np

reader = ASTReader()
path_ast = "ast"


kits = []
for dir_kit in os.listdir(path_ast): # List all kits
    if os.path.isdir(os.path.join(path_ast, dir_kit)):
        kits.append(dir_kit)
kits = sorted(kits)




    

# 1. Fichiers
## 1.1 Clones paramétriques

On va lister les fichiers ayant des vecteurs identiques, en limitant notre recherhe aux AST de plus de 100 noeuds

In [22]:
Selec=list()
Selec_file=list()
for kit in kits:
    kit_files = []
    for root, dirs, files in os.walk(os.path.join(path_ast, kit)): # Find all files for this kit
        kit_files.extend((os.path.join(root, file) for file in files))
    
    for file in kit_files:
        
        ast = reader.read_ast(file)
        vector = ast.vectorize()
        length=vector.sum()
        #print(length)
        if (length>100): #On limite la sélection aux AST de plus de 100 noeuds
                Selec.append(vector)
                Selec_file.append(file)
        



On utilise le module numpy, avec la fonction unique, qui va nous retourner une matrice de vecteurs uniques, accompagnée de leur nombre d'occurences.

In [23]:


uniq,inv,ret=np.unique(Selec,axis=0, return_counts=True,return_inverse=True)



print("----------------------------------------------------------------")


print("Nombre d'AST sup à 100 noeuds :")
print( len(Selec))
print("Nombre d'AST avec vecteurs uniques :") 
print( len(uniq))
print("Plus grand nombre d'occurence d'un vecteur")
print(ret.max())

print("liste des fichiers produisant ce vecteur")
max_vec_index=np.where(inv==np.where(ret==47)[0])
for i in max_vec_index[0]:
    print(Selec_file[i])


----------------------------------------------------------------
Nombre d'AST sup à 100 noeuds :
899
Nombre d'AST avec vecteurs uniques :
538
Plus grand nombre d'occurence d'un vecteur
47
liste des fichiers produisant ce vecteur
ast/2099/dhl/def/start/index.php.ast.json.gz
ast/2099/dhl/def/info/index.php.ast.json.gz
ast/2099/dhl/def/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c5489ea1c9eee2d64ecf7/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c5489ea1c9eee2d64ecf7/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c5489ea1c9eee2d64ecf7/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/1f7ebc62d330d7e48473470961169eb4/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/1f7ebc62d330d7e48473470961169eb4/info/index.php.ast.json.gz
a

Le plus grand nombre d'occurence du même vetcur est donc de 47. On remarque que la majorité des vecteurs retournées viennent du même kit, a1b2c3.<br><br>
Afin d'éviter de retrouver des vecteurs similaires du même kit, qui peuvent être la conséquence d'une duplication de code dans le même fichier, on va ignorer les vecteurs du même kit lors de la prise en compte de la similarité. 

In [24]:
#avoiding file from same kit
uniq,inv,ret=np.unique(Selec,axis=0, return_counts=True,return_inverse=True)
for i,vec in enumerate(uniq):
    count=0
    kit_used=[]
    for index in np.where(inv==i)[0]:
       
        kit_num=int(Selec_file[index].split("/")[1])
        
        if kit_num not in kit_used:
            kit_used.append(kit_num)
            count=count+1
    ret[i]=count

    
print(ret.max()) # on a beaucoup moins de score et des égalites
print("liste des fichiers produisant ce vecteur")

for ind in np.where(ret==ret.max())[0]:
    print("----------") # il y a des égalités sur le nombre de vecteurs identiques, on les sépare
    max_vec_ind=np.where(inv==ind)
    for i in max_vec_ind[0]:
        print(Selec_file[i])

    




7
liste des fichiers produisant ce vecteur
----------
ast/0415/M&T/darkx/anti2.php.ast.json.gz
ast/0485/we logz sure/anti/anti2.php.ast.json.gz
ast/1110/submit/prevents/anti2.php.ast.json.gz
ast/1442/web/auth/Bots/anti2.php.ast.json.gz
ast/3135/hunting/antibot/anti2.php.ast.json.gz
ast/3135/hunting/antibot/anti10.php.ast.json.gz
ast/3662/usps-shippment/bots/anti2.php.ast.json.gz
ast/3716/gruposantander/xJOESTAR/anti2.php.ast.json.gz
ast/3716/gruposantander/xJOESTAR/anti10.php.ast.json.gz
----------
ast/0415/M&T/darkx/anti8.php.ast.json.gz
ast/0485/we logz sure/anti/anti8.php.ast.json.gz
ast/1110/submit/prevents/anti8.php.ast.json.gz
ast/1110/submit/prevents/kiks8.php.ast.json.gz
ast/1442/web/auth/Bots/anti8.php.ast.json.gz
ast/3135/hunting/antibot/anti16.php.ast.json.gz
ast/3135/hunting/antibot/anti8.php.ast.json.gz
ast/3662/usps-shippment/bots/anti8.php.ast.json.gz
ast/3716/gruposantander/xJOESTAR/anti16.php.ast.json.gz
ast/3716/gruposantander/xJOESTAR/anti8.php.ast.json.gz
----------

## 1.2 Fichiers similaires

In [25]:
#1.2
def compare_string_list(A,B):
    if len(A)!=len(B):
        return False
    for i in range(len(A)):
        if A[i]!=B[i]:
            return False
    return True

uniq,inv,ret=np.unique(Selec,axis=0, return_counts=True,return_inverse=True)

groups=[]

for x in uniq:
    res=[]
    for i,k in enumerate(Selec):
        if np.abs(x-k).sum() <= 0.3 * np.sum(k): #On considère que deux vecteurs sont similaires si leur distance est inférieure à 30%
            res.append(Selec_file[i])
    groups.append(res)
    
max_index=[0]
max_val=0
for i,x in enumerate(groups):
    if len(x)>max_val:
        max_val=len(x)
        max_index=[i]
    if len(x) == max_val:
        max_index.append(i)

print(max_val)
print("liste des groupes de fichies produisant ce(s) vecteur(s)")

for file in groups[max_index[0]]:
        print(file)
for ind_max in max_index[1:]:
    
    if not compare_string_list(groups[max_index[0]],groups[ind_max]):
        print("----------------------\nSeparation between different vecteurs\n----------------------")
    
        for file in groups[ind_max]:
            print(file)


81
liste des groupes de fichies produisant ce(s) vecteur(s)
ast/2099/dhl/def/start/index.php.ast.json.gz
ast/2099/dhl/def/vbv/index.php.ast.json.gz
ast/2099/dhl/def/info/index.php.ast.json.gz
ast/2099/dhl/def/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/vbv/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c5489ea1c9eee2d64ecf7/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c5489ea1c9eee2d64ecf7/vbv/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c5489ea1c9eee2d64ecf7/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c5489ea1c9eee2d64ecf7/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/1f7ebc62d330d7e48473470961169eb4/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/1f7ebc62d330d7e48473470961169

# 2. Fragments

## 2.1 Clones paramétriques

In [26]:
#part 2
#computing of fragment vector



Selec=list()
Selec_file=list()
for kit in kits:
    kit_files = []
    for root, dirs, files in os.walk(os.path.join(path_ast, kit)): # Find all files for this kit
        kit_files.extend((os.path.join(root, file) for file in files))
    
    for file in kit_files:
        ast = reader.read_ast(file)
        nodes=ast.get_node_ids()
        e_node=[ast.get_root()]
        
        for n in nodes:
            if ast.get_type(n) == "MethodStatement" or ast.get_type(n) == "FunctionStatement":
                p=ast.get_parents(n)
                ast.remove_edge(p[0],n)
                e_node.append(n)
        Selec.append([ast,e_node])
        Selec_file.append(file)

vec=[]
vec_name=[]
for index_kit,f in enumerate(Selec):
    for entry in f[1]:
        nodes=f[0].dfs(entry)
        if len(nodes)>10:
            v = np.zeros(len(f[0].types), dtype=int)
            for node in nodes:
                if f[0].get_type(node) not in f[0].types:
                    print(f"WARN: Type {f[0].get_type(node)} unknown. Skip node {node}..")
                    continue
                v[f[0].types.index(f[0].get_type(node))] += 1
            
            kit_num=Selec_file[index_kit].split("/")[1]
            vec.append(v)
            image=f[0].get_image(entry)
            if not image:
                image="entry"
            vec_name.append(kit_num+"/"+image)
    
print(vec[0:2])
print(vec_name[0:2])
        

        
        
        
        

[array([0, 0, 0, 0, 7, 9, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0]), array([ 0,  0,  0,  0,  7,  9,  0,  0,  0,  0, 12,  5,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  7,  1,  0,  0,  0,  0,
        8,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
        1,  0,  0,  4,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  2, 20,  0,  0,  1])]
['0009/e

In [27]:
# computing of groups
#2.1
uniq,inv,ret=np.unique(vec,axis=0, return_counts=True,return_inverse=True)
for i in range(len(uniq)):
    count=0
    kit_used=[]
    for index in np.where(inv==i)[0]:
       
        kit_num=int(vec_name[index].split("/")[0])
        
        if kit_num not in kit_used:
            kit_used.append(kit_num)
            count=count+1
    ret[i]=count

print("Nombre de duplication maximum:") 
print(ret.max()) # on a beaucoup moins de score et des égalites
print("liste des fonctions/méthodes produisant ce vecteur")

for ind in np.where(ret==ret.max())[0]:
    print("----------") # il y a des égalités sur le nombre de vecteurs identiques, ont les sépares
    max_vec_ind=np.where(inv==ind)
    for i in max_vec_ind[0]:
        print(vec_name[i])



Nombre de duplication maximum:
12
liste des fonctions/méthodes produisant ce vecteur
----------
0009/recurse_copy
0229/recurse_copy
1110/recurse_copy
1651/recurse_copy
2070/recurse_copy
2099/dublicate
2110/dublicate
3180/recurse_copy
3548/recurse_copy
3676/recurse_copy
4218/recurse_copy
4417/dublicate


## 2.2 Fragments Similaires

In [28]:
#2.2


uniq,inv,ret=np.unique(vec,axis=0, return_counts=True,return_inverse=True)

groups=[]

for j,x in enumerate(uniq):
    res=[]
    kit_used=[]
    for i,k in enumerate(vec):
        k_kit_num=int(vec_name[i].split("/")[0])
        if k_kit_num not in kit_used:
           
            if np.abs(x-k).sum() <= 0.1 * np.sum(k):
                kit_used.append(k_kit_num)
                res.append(vec_name[i])
    groups.append(res)
    
max_index=[]
max_val=0
for i,x in enumerate(groups):
    if len(x)>max_val:
        max_val=len(x)
        max_index=[i]
    if len(x) == max_val:
        max_index.append(i)


print("Nombre de similarités maximum:") 

print(max_val)

print("liste des fonctions/méthodes produisant ce(s) vecteur(s)")

for file in groups[max_index[0]]:
        print(file)
for ind_max in max_index[1:]:
    
    if not compare_string_list(groups[max_index[0]],groups[ind_max]):
        print("----------------------\nSeparation between different vecteurs\n----------------------")
    
        for file in groups[ind_max]:
            print(file)




Nombre de similarités maximum:
16
liste des fonctions/méthodes produisant ce(s) vecteur(s)
0109/getOS
0229/get_user_os
0406/getOS
0415/getOs
0431/getOS
0481/XB_OS
1033/getOs
1110/get_user_os
1442/XB_OS
1651/get_user_os
1926/getOS
2866/XB_OS
3148/X_OS
3716/getOs
3858/getOs
4273/XB_OS


# 3. Kits paramétriques

In [46]:
#part 3

Selec=list()
Selec_file=list()
for kit in kits:
    kit_files = []
    for root, dirs, files in os.walk(os.path.join(path_ast, kit)): # Find all files for this kit
        kit_files.extend((os.path.join(root, file) for file in files))
    
    for file in kit_files:
        ast = reader.read_ast(file)
        nodes=ast.get_node_ids()
        e_node=[ast.get_root()]
        
        for n in nodes:
            if ast.get_type(n) == "MethodStatement" or ast.get_type(n) == "FunctionStatement":
                p=ast.get_parents(n)
                ast.remove_edge(p[0],n)
                e_node.append(n)
        Selec.append([ast,e_node])
        Selec_file.append(file)

vec=[]
vec_name=[]
for index_kit,f in enumerate(Selec):
    for entry in f[1]:
        nodes=f[0].dfs(entry)
        v = np.zeros(len(f[0].types), dtype=int)
        for node in nodes:
            if f[0].get_type(node) not in f[0].types:
                print(f"WARN: Type {f[0].get_type(node)} unknown. Skip node {node}..")
                continue
            v[f[0].types.index(f[0].get_type(node))] += 1
            
        kit_num=Selec_file[index_kit].split("/")[1]
        vec.append(v)
        image=f[0].get_image(entry)
        if not image:
            image="entry"
        vec_name.append(kit_num+"/"+image)

def somme_list(A,B):
    res=[]
    for i in range(len(A)):
        res.append(A[i]+B[i])
    return res

kit_vec=[[0 for j in range(len(vec[0]))] for i in range(len(kits))]

index=0
kit_index=int(vec_name[0].split("/")[0])
kit_vec_name=[kit_index]

for i,vector in enumerate(vec):
    kit_num=int(vec_name[i].split("/")[0])
    if kit_num != kit_index:
        index=index+1
        kit_index=kit_num
        kit_vec_name.append(kit_num)
    

    kit_vec[index]=somme_list(kit_vec[index],vector)
    

print(len(kit_vec))
print(len(kit_vec_name))
print(kit_vec[0:2])
print(kit_vec_name)



99
99
[[0, 0, 0, 0, 14, 18, 0, 0, 0, 0, 24, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 14, 1, 0, 0, 0, 7, 15, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 3, 4, 0, 0, 7, 11, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 41, 0, 0, 1], [0, 0, 0, 0, 1533, 1907, 0, 0, 62, 273, 1291, 341, 0, 0, 25, 34, 9, 0, 5, 0, 39, 25, 25, 0, 997, 858, 858, 858, 1, 4, 0, 2, 0, 2, 0, 841, 0, 10, 4, 0, 0, 31, 0, 4, 18, 502, 10, 2, 0, 0, 863, 1177, 0, 4, 58, 60, 1, 2, 0, 4, 4, 0, 0, 153, 0, 0, 0, 34, 3, 186, 4, 0, 0, 0, 109, 0, 0, 0, 0, 96, 129, 19, 0, 4, 0, 0, 0, 1, 5, 0, 4, 6, 0, 5, 104, 0, 877, 47, 1, 125, 0, 0, 10, 119, 39, 215, 0, 0, 3122, 348, 9, 16, 0, 34, 5, 0, 0, 69, 4, 0, 0, 0, 94, 2885, 0, 2, 3]]
[9, 40, 63, 109, 173, 214, 229, 266, 325, 398, 406, 415, 431, 481, 485, 556, 809, 844, 905, 926, 1023, 1033, 1110, 1134, 1217, 1244, 1352, 135

In [52]:
#part3 - compute the groups

uniq,inv,ret=np.unique(kit_vec,axis=0, return_counts=True,return_inverse=True)



print("----------------------------------------------------------------")
#print(inv)


print("Nombre d'AST avec vecteurs uniques :") 
print( len(uniq))
print("Plus grand nombre d'occurence d'un vecteur")
print(ret.max())

print("liste des kits produisant ce vecteur")
max_vec_index=np.where(inv==np.where(ret==ret.max()))
for i in max_vec_index[1]:
    print(kit_vec_name[i])
    
# Ces trois kits sont du style amazon



----------------------------------------------------------------
Nombre d'AST avec vecteurs uniques :
93
Plus grand nombre d'occurence d'un vecteur
3
liste des kits produisant ce vecteur
481
2866
4273
